<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Download-embeddings-from-PostgresSQL" data-toc-modified-id="Download-embeddings-from-PostgresSQL-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Download embeddings from PostgresSQL</a></span></li><li><span><a href="#Try-to-load-embeddings" data-toc-modified-id="Try-to-load-embeddings-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Try to load embeddings</a></span></li></ul></div>

In [ ]:
from esper.stdlib import *
from esper.prelude import *
import os
import struct
import json

def decode(features):
    return json.loads(features.tobytes().decode())

# Download embeddings from PostgresSQL

In [ ]:
EMBS_DIR = '/app/data/embs/'
IDS_FILE = os.path.join(EMBS_DIR, 'face_ids.bin')
EMBS_FILE = os.path.join(EMBS_DIR, 'face_embs.bin')
ENDIAN = 'little'
DIMENSIONS = 128

if not os.path.exists(EMBS_DIR):
    os.makedirs(EMBS_DIR)

with open(IDS_FILE, 'wb') as f_ids, open(EMBS_FILE, 'wb') as f_embs:
    for i, x in enumerate(FaceFeatures.objects.all().values(
                'face__id', 'features'
             ).order_by('face__id')):
        if i % 10000 == 0:
            print('Wrote {} embeddings'.format(i))
        fid = x['face__id']
        emb = decode(x['features'])
        assert len(emb) == DIMENSIONS, 'Incorrect dimensions: {} != {}'.format(
            len(emb), DIMENSIONS)
        f_ids.write(fid.to_bytes(8, byteorder=ENDIAN))
        for v in emb:
            f_embs.write(struct.pack('f', v))
    print('Done! ({} embeddings written)'.format(i + 1))

# Try to load embeddings

In [ ]:
import esper.face_embeddings as face_embed